<a href="https://colab.research.google.com/github/Ankitattri16/recommendation/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data collection and Pre-Processing

In [ ]:
def fix_unmatched_quotes(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    corrected_lines = []
    for line in lines:
        if line.count('"') % 2 != 0:
            # Add a closing quote if the count of quotes is odd
            line = line.strip() + '"\n'
        corrected_lines.append(line)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(corrected_lines)

fix_unmatched_quotes("/content/movies.csv")
movies_data = pd.read_csv("/content/movies.csv")


In [ ]:
#printing the first five rows of the dataframe
movies_data. head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#number of rows and columns in the data frame
movies_data . shape

(2158, 24)

In [ ]:
#selecting the releavent features for recommendation
selected_features = ['genre','keywords','tagline','director']
print(selected_features)

['genre', 'keywords', 'tagline', 'director']


In [ ]:
combined_features = movies_data['genres']+''+movies_data['keywords']+''+movies_data['tagline']+''+movies_data['cast']+''+movies_data['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
2153    Comedy Romancebrother sister relationship marr...
2154    Action Crime Drama Thrillerpolice los angelesT...
2155    Comedy Action Science Fictionend of the world ...
2156    Action Adventure Crime Family Mysterycaliforni...
2157                                                  NaN
Length: 2158, dtype: object


In [ ]:
# Check for NaN values
print("NaN values before handling:")
print(combined_features.isna().sum())

# Handle NaN values by replacing with an empty string
combined_features = combined_features.fillna('')

# Check for NaN values after handling
print("NaN values after handling:")
print(combined_features.isna().sum())

# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the data
feature_vectors = vectorizer.fit_transform(combined_features)

# Print the feature vectors
print(feature_vectors.toarray())

NaN values before handling:
185
NaN values after handling:
0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
print(feature_vectors)

  (0, 1791)	0.1725883371319682
  (0, 10212)	0.26440477592566114
  (0, 8139)	0.1605704385587145
  (0, 7111)	0.25097748251464375
  (0, 11470)	0.17701033338391925
  (0, 13144)	0.19767969031274837
  (0, 11031)	0.20506926311779727
  (0, 10568)	0.21459607940520314
  (0, 13638)	0.20795071275569002
  (0, 13466)	0.22291855278076572
  (0, 10583)	0.15651036700462492
  (0, 9091)	0.25097748251464375
  (0, 8891)	0.09090795772645784
  (0, 13454)	0.12301041353307703
  (0, 11946)	0.07669939080062678
  (0, 11201)	0.26440477592566114
  (0, 2323)	0.24145066622723793
  (0, 13018)	0.13518790653680876
  (0, 11274)	0.32560761838392954
  (0, 5012)	0.16962076057887035
  (0, 2218)	0.24145066622723793
  (0, 4588)	0.26440477592566114
  (0, 10722)	0.09696325104720852
  (0, 4372)	0.11108325699317821
  (0, 227)	0.08270281230437844
  :	:
  (2156, 6873)	0.26302208080396966
  (2156, 8546)	0.26302208080396966
  (2156, 11978)	0.24966500493358457
  (2156, 4787)	0.24966500493358457
  (2156, 11843)	0.24018800878410826
  (215

In [ ]:
#getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.0659586  0.01270753 ... 0.04194214 0.01883152 0.        ]
 [0.0659586  1.         0.02586984 ... 0.04122059 0.02383096 0.        ]
 [0.01270753 0.02586984 1.         ... 0.00526895 0.06573303 0.        ]
 ...
 [0.04194214 0.04122059 0.00526895 ... 1.         0.00498701 0.        ]
 [0.01883152 0.02383096 0.06573303 ... 0.00498701 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
print(similarity.shape)

(2158, 2158)


In [ ]:
#getting the movie name from the user
movie_name = input('Enter your favourite movie name:')

Enter your favourite movie name:ironman


In [ ]:
#creating a list with all the movie names given in the data set
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
#finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Iron Man', 'Gridiron Gang']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [ ]:
#find the index of the movie
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)


68


In [ ]:
#getting list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.013661486482143808), (1, 0.03530319724406544), (2, 0.005145467408529855), (3, 0.005431092765892404), (4, 0.01306929881487099), (5, 0.005035024820347633), (6, 0.05780341661916248), (7, 0.14104865755217136), (8, 0.0), (9, 0.031031958774738178), (10, 0.0), (11, 0.004497770429534954), (12, 0.0), (13, 0.004720937893807882), (14, 0.03984356850330205), (15, 0.0), (16, 0.17949520859346263), (17, 0.004808167653111434), (18, 0.013226588103473439), (19, 0.004337323786820451), (20, 0.05904378817169118), (21, 0.004375700786293635), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.15423293299920454), (27, 0.012462096307755126), (28, 0.05643782821284146), (29, 0.005187393054322929), (30, 0.060392806085069924), (31, 0.21454087923363185), (32, 0.023074301409517883), (33, 0.11399018792247922), (34, 0.0), (35, 0.029750083858277456), (36, 0.02747438032350352), (37, 0.0), (38, 0.08781011455467204), (39, 0.07666731464451912), (40, 0.0), (41, 0.01298223611890692), (42, 0.0), (43, 0.02769833160241217

In [ ]:
len(similarity_score)

2158

In [ ]:
#sorting the movies based on their similariy score
sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse = True)
print(sorted_similar_movies)

[(68, 1.0000000000000002), (79, 0.23211575599935208), (31, 0.21454087923363185), (16, 0.17949520859346263), (26, 0.15423293299920454), (7, 0.14104865755217136), (353, 0.11614054978431115), (33, 0.11399018792247922), (46, 0.11088450565868463), (1210, 0.10581481744085684), (511, 0.10273872096697592), (954, 0.10106576643085935), (203, 0.10015128859673361), (1406, 0.09753376318593693), (174, 0.09553847722831227), (882, 0.09433729584521312), (85, 0.09324450371139793), (64, 0.0909263792389573), (307, 0.08953392269556068), (2063, 0.08913535284042004), (101, 0.08872697085502994), (38, 0.08781011455467204), (1282, 0.08664455316588396), (356, 0.08246429443348761), (788, 0.08214268971264951), (783, 0.0816232658317333), (205, 0.08161313996226646), (1972, 0.07745650137603856), (1553, 0.07666951480340121), (39, 0.07666731464451912), (182, 0.0755055091834003), (661, 0.07436328093688183), (2066, 0.073854838552221), (1664, 0.07370006687014793), (1654, 0.07266181764943963), (1311, 0.07252585525470573), 

In [ ]:
print('Movies suggested for you:\n')
i = 1
for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]['title'].values[0]
    if i < 30:
        print(i, '.', title_from_index)
        i += 1


Movies suggested for you:

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . The Avengers
5 . Captain America: Civil War
6 . Avengers: Age of Ultron
7 . Tropic Thunder
8 . X-Men: The Last Stand
9 . X-Men: Days of Future Past
10 . Gothika
11 . X-Men
12 . The Judge
13 . X2
14 . The Nativity Story
15 . The Incredible Hulk
16 . Lucky You
17 . Captain America: The Winter Soldier
18 . X-Men: Apocalypse
19 . The Expendables 3
20 . Arlington Road
21 . X-Men: First Class
22 . The Amazing Spider-Man 2
23 . The Hunting Party
24 . Sherlock Holmes
25 . Deadpool
26 . Mortdecai
27 . Sherlock Holmes: A Game of Shadows
28 . The Men Who Stare at Goats
29 . Se7en


Movie Recommendation system


In [ ]:
movie_name = input('Enter your favourite movie name:')

list_of_all_titles = movies_data['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
close_match = find_close_match[0]
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse = True)
print('Movies suggested for you:\n')
i = 1
for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]['title'].values[0]
    if i < 30:
        print(i, '.', title_from_index)
        i += 1

Enter your favourite movie name:batman
Movies suggested for you:

1 . Batman
2 . Batman Returns
3 . Batman & Robin
4 . Batman Begins
5 . Man of Steel
6 . The Dark Knight Rises
7 . I Dreamed of Africa
8 . Bedazzled
9 . Reds
10 . Mars Attacks!
11 . Jonah Hex
12 . The Dark Knight
13 . Something's Gotta Give
14 . A History of Violence
15 . Superman III
16 . Blood and Wine
17 . 8 Mile
18 . Superman
19 . Cellular
20 . Multiplicity
21 . Superman II
22 . Planet of the Apes
23 . Catwoman
24 . Batman Forever
25 . Spotlight
26 . The 6th Day
27 . The American President
28 . Megiddo: The Omega Code 2
29 . How Do You Know
